In [1]:
using CUDA

In [5]:
A = rand(31,3) |> cu
B = similar(A) |> cu

function test!(A,B)

    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y 

    norm = @cuDynamicSharedMem(Float32,(i,k))

    if i <= size(A,1) && k <= size(A,2)
        norm[i,k] = A[i,k]
        B[i,k] = 2*norm[i,k]
    end

    return nothing
end

threads = (32,3)
blocks = cld.(size(A),threads)
@cuda(
    threads = threads,
    blocks = blocks,
    shmem=prod(threads.+2)*sizeof(Float32),
    test!(A,B)
)
display(A)
display(B)

31×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.323591    0.519328   0.288089
 0.163718    0.387013   0.322694
 0.347982    0.76562    0.21254
 0.770782    0.991324   0.03531
 0.466748    0.604893   0.244846
 0.901701    0.525085   0.754386
 0.721079    0.0815532  0.78039
 0.569114    0.121778   0.451746
 0.00017292  0.972944   0.0841276
 0.913255    0.901709   0.480179
 ⋮                      
 0.682397    0.605486   0.835493
 0.36678     0.602152   0.438748
 0.151793    0.9244     0.65088
 0.973446    0.0836412  0.434435
 0.550676    0.0152327  0.1563
 0.117456    0.588559   0.988195
 0.266266    0.940621   0.763749
 0.0295066   0.39672    0.557986
 0.479053    0.164226   0.279505

31×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.647181     1.03866    0.576177
 0.327435     0.774026   0.645388
 0.695965     1.53124    0.425079
 1.54156      1.98265    0.0706201
 0.933495     1.20979    0.489692
 1.8034       1.05017    1.50877
 1.44216      0.163106   1.56078
 1.13823      0.243556   0.903491
 0.000345839  1.94589    0.168255
 1.82651      1.80342    0.960358
 ⋮                       
 1.36479      1.21097    1.67099
 0.73356      1.2043     0.877495
 0.303586     1.8488     1.30176
 1.94689      0.167282   0.86887
 1.10135      0.0304655  0.312599
 0.234912     1.17712    1.97639
 0.532532     1.88124    1.5275
 0.0590132    0.79344    1.11597
 0.958106     0.328452   0.55901